In [1]:
import requests
import hashlib
import os.path
import datetime
import re
import numpy
import pandas as pd
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
from dateutil.parser import parse as parsedate
from datetime import date

FILE_PADS = "sites.tsv"
FILE_TEMP2 = "sites.tmp"
FILE_DATA = "launchdata.tsv"
FILE_TEMP = "launchdata.tmp"

country = ["SUA", "Rusia", "China", "Europa", "Japonia", "India", "Iran", "Israel", "Coreea de Sud", "Coreea de Nord", "Brazilia"]
%store country

today = datetime.date.today()
curr_year = today.year

Stored 'country' (list)


In [2]:
def update_GCAT():
	
	if os.path.isfile(FILE_TEMP): os.remove(FILE_TEMP)
	
	print('Checking for GCAT update...')
	url = 'https://planet4589.org/space/gcat/tsv/launch/launch.tsv'
	response = requests.get(url, allow_redirects=True)
	open(FILE_TEMP,"wb").write(response.content)

	r = requests.head(url)
	url_time = r.headers['last-modified']
	url_date = parsedate(url_time)

	md5_hash = hashlib.md5
	with open(FILE_TEMP,"rb") as f:
		bytes = f.read()
		readable_hash = hashlib.md5(bytes).hexdigest()

	if os.path.isfile(FILE_DATA):
		with open(FILE_DATA,"rb") as g:
			bytes = g.read()
			readable_hash2 = hashlib.md5(bytes).hexdigest()
			if readable_hash == readable_hash2: 
				os.remove(FILE_TEMP)
				print ("No updates found!")
			else: 
				os.remove(FILE_DATA)
				os.rename(FILE_TEMP,FILE_DATA)
				print ("New GCAT version installed, date:",url_date)
	else:
		os.rename(FILE_TEMP,FILE_DATA)
		print ("No old GCAT found, fetched latest version from date",url_date)
		

In [3]:
def update_PADS():

	if os.path.isfile(FILE_TEMP2): os.remove(FILE_TEMP2)

	print('Checking for PADS update...')
	url = 'https://planet4589.org/space/gcat/tsv/tables/sites.tsv'
	response = requests.get(url, allow_redirects=True)
	open(FILE_TEMP2,"wb").write(response.content)


	r = requests.head(url)
	url_time = r.headers['last-modified']
	url_date = parsedate(url_time)

	md5_hash = hashlib.md5
	with open(FILE_TEMP2,"rb") as f:
		bytes = f.read()
		readable_hash = hashlib.md5(bytes).hexdigest()

	if os.path.isfile(FILE_PADS):
		with open(FILE_PADS,"rb") as g:
			bytes = g.read()
			readable_hash2 = hashlib.md5(bytes).hexdigest()
			if readable_hash == readable_hash2: 
				os.remove(FILE_TEMP2)
				print ("No updates found!")
			else: 
				os.remove(FILE_PADS)
				os.rename(FILE_TEMP2,FILE_PADS)
				print ("New PADS version installed, date:",url_date)
	else:
		os.rename(FILE_TEMP2,FILE_PADS)
		print ("No old PADS found, fetched latest version from date",url_date)

In [4]:
if not os.path.isfile(FILE_DATA): update_GCAT()
if not os.path.isfile(FILE_PADS): update_PADS()

In [5]:
# Quick and dirty fix, since I consider 2022-U02 to be a proper failed launch deserving the F tag
# To be removed if this changes upstream

fin = open(FILE_DATA, "rt")
data = fin.read()
data = data.replace('2022-U02', '2022-F0x')
fin.close()
fin = open(FILE_DATA, "wt")
fin.write(data)
fin.close()

In [6]:
df = pd.read_csv(FILE_DATA, sep='\t', header=None, skiprows=lambda x: x<2, names=['Launch_Tag', 'Launch_JD', 'Launch_Date', 'LV_Type', 'Variant', 'Flight_ID', 'Flight', 'Mission', 'FlightCode', 'Platform', 'Launch_Site', 'Launch_Pad', 'Ascent_Site', 'Ascent_Pad', 'Apogee', 'Apoflag', 'Range', 'RangeFlag', 'Dest', 'Agency', 'Launch_Code', 'Group', 'Category', 'LTCite', 'Cite', 'Notes'])
pads = pd.read_csv(FILE_PADS, sep='\t', header=None, skiprows=lambda xx: xx<2, names=['Site', 'Code', 'UCode', 'Type', 'StateCode', 'TStart', 'TStop', 'ShortName', 'Name', 'Location', 'Longitude', 'Latitude', 'Error', 'Parent', 'ShortEName', 'EName', 'Group', 'UName'])

## Filtering suborbital launches
df = df[df["Launch_Tag"].str.contains("-S") == False]
df = df[df["Launch_Tag"].str.contains("-A") == False]
df = df[df["Launch_Tag"].str.contains("-M") == False]
df = df[df["Launch_Tag"].str.contains("-W") == False]
df = df[df["Launch_Tag"].str.contains("-Y") == False]
df = df[df["Launch_Tag"].str.contains("-U") == False]
df = df[df["Launch_Tag"].str.contains("2014-000") == False]

#ldata = df.to_numpy()
pdata = pads.to_numpy()

ldata_rows = len(df.index)
pdata_rows = len(pads.index)

i = 0
j = 0
ccountry = []
status =[] 
llocation = []

ldata = df.to_numpy()

while i<ldata_rows:
	j = 0
	while j<pdata_rows:
		if ldata[i][10].strip() == pdata[j][0].strip(): 
			tmp_country = pdata[j][4]
			tmp_location = pdata[j][8]
			if tmp_country == "DZ": tmp_country = "EU"
			if tmp_country == "J": tmp_country = "JP"
			if tmp_country == "GUF": tmp_country = "EU"
			if tmp_country == "AU": tmp_country = "EU"
			if tmp_country == "NZ": tmp_country = "US"
			if tmp_country == "TTPI": tmp_country = "US"
			if tmp_country == "ESCN": tmp_country = "US"
			if tmp_country == "KE": tmp_country = "US"
			if tmp_country == "KI": tmp_country = "RU"
			if tmp_country == "SU": tmp_country = "RU"
			ccountry.append(tmp_country)
			llocation.append(tmp_location)
		j+=1     
	tmp_status = ldata[i][0]
	if "-F" in str(tmp_status): 
		status.append("F")
	else:
		status.append("S")
	i+=1
df['Country'] = ccountry
df['Status'] = status
df['Location'] = llocation

i = 0
while i<ldata_rows:
	if " Jan " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Jan ","-01-")
	if " Feb " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Feb ","-02-")
	if " Mar " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Mar ","-03-")
	if " Apr " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Apr ","-04-")
	if " May " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" May ","-05-")
	if " Jun " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Jun ","-06-")
	if " Jul " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Jul ","-07-")
	if " Aug " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Aug ","-08-")
	if " Sep " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Sep ","-09-")
	if " Oct " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Oct ","-10-")
	if " Nov " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Nov ","-11-")
	if " Dec " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace(" Dec ","-12-") 
	if "- " in ldata[i][2]: ldata[i][2] = ldata[i][2].replace("- ","-0")
	if "Delta 4H" in ldata[i][3]: ldata[i][3] = ldata[i][3].replace("Delta 4H","Delta 4 Heavy")
	if "Soyuz-2-1A" in ldata[i][3]: ldata[i][3] = ldata[i][3].replace("Soyuz-2-1A","Soyuz-2.1a")
	if "Soyuz-2-1B" in ldata[i][3]: ldata[i][3] = ldata[i][3].replace("Soyuz-2-1B","Soyuz-2.1b")
	if "Soyuz-2-1V" in ldata[i][3]: ldata[i][3] = ldata[i][3].replace("Soyuz-2-1V","Soyuz-2.1v")
	if "GSLV Mk II" == ldata[i][3].strip(): ldata[i][3] = ldata[i][3].replace("GSLV Mk II","GSLV Mk. 2")
	if "GSLV Mk III" in ldata[i][3]: ldata[i][3] = ldata[i][3].replace("GSLV Mk III","GSLV Mk. 3")
	i+=1

ldata = df.to_numpy()

In [7]:
def locations():
    field_names = ["Centru", "Denumire", "Țara"]
    data = []
    i = 0
    list_locations = pd.unique(df['Launch_Site'])

    i = 0
    while i < pdata_rows:
        if pdata[i][0] in list_locations:
            data.append([pdata[i][0], pdata[i][8], pdata[i][4]])
        i+=1
    df_locations = pd.DataFrame(data, columns=field_names)
    return(df_locations.to_markdown(index=False))
    
list_legend = locations()
%store list_legend

Stored 'list_legend' (str)


In [8]:
def succ_country(year):
 
	l_succ_country = [0] * 12
	i = 0
    
	while i<ldata_rows:
		if str(year) in str(ldata[i][2][0:7]) and "F" not in str(ldata[i][0]): 
			l_succ_country[11]+=1
			if ldata[i][26] == "US": l_succ_country[0]+=1
			if ldata[i][26] == "RU": l_succ_country[1]+=1
			if ldata[i][26] == "CN": l_succ_country[2]+=1
			if ldata[i][26] == "EU": l_succ_country[3]+=1
			if ldata[i][26] == "JP": l_succ_country[4]+=1
			if ldata[i][26] == "IN": l_succ_country[5]+=1
			if ldata[i][26] == "IR": l_succ_country[6]+=1
			if ldata[i][26] == "IL": l_succ_country[7]+=1
			if ldata[i][26] == "KR": l_succ_country[8]+=1
			if ldata[i][26] == "KP": l_succ_country[9]+=1
			if ldata[i][26] == "BR": l_succ_country[10]+=1
		i+=1
	return(l_succ_country)

In [9]:
def fail_country(year):
    
	l_fail_country = [0] * 12
	i = 0
    
	while i<ldata_rows:
		if str(year) in str(ldata[i][2][0:7]) and "F" in str(ldata[i][0]): 
			l_fail_country[11]+=1
			if ldata[i][26] == "US": l_fail_country[0]+=1
			if ldata[i][26] == "RU": l_fail_country[1]+=1
			if ldata[i][26] == "CN": l_fail_country[2]+=1
			if ldata[i][26] == "EU": l_fail_country[3]+=1
			if ldata[i][26] == "JP": l_fail_country[4]+=1
			if ldata[i][26] == "IN": l_fail_country[5]+=1
			if ldata[i][26] == "IR": l_fail_country[6]+=1
			if ldata[i][26] == "IL": l_fail_country[7]+=1
			if ldata[i][26] == "KR": l_fail_country[8]+=1
			if ldata[i][26] == "KP": l_fail_country[9]+=1
			if ldata[i][26] == "KP": l_fail_country[9]+=1
			if ldata[i][26] == "BR": l_fail_country[10]+=1
		i+=1
	return(l_fail_country)
    

In [10]:
# număr total lansări de succes pe țări, exportat ca array
def total_succ_country():
    i = 1957
    j = 0
    tsuc_country = [0] * 12
    while i<=curr_year:
        j = 0
        while j<12:
            tsuc_country[j] = tsuc_country[j] + succ_country(i)[j]
            j+=1
        i+=1
    return(tsuc_country)
ex_total_succ_country = total_succ_country()
%store ex_total_succ_country

Stored 'ex_total_succ_country' (list)


In [11]:
# număr total eșecuri pe țări, exportat ca array
def total_fail_country():
    i = 1957
    j = 0
    tfail_country = [0] * 12
    while i<=curr_year:
        j = 0
        while j<12:
            tfail_country[j] = tfail_country[j] + fail_country(i)[j]
            j+=1
        i+=1
    return(tfail_country)
ex_total_fail_country = total_fail_country()
%store ex_total_fail_country

Stored 'ex_total_fail_country' (list)


In [12]:
def total_tot_country():
    i = 0
    ttot_country = [0] * 12
    while i < 12:
        ttot_country[i] = ex_total_succ_country[i] + ex_total_fail_country[i]
        i+=1
    return(ttot_country)
ex_total_tot_country = total_tot_country()
%store ex_total_tot_country

Stored 'ex_total_tot_country' (list)


In [13]:
def preambul(year):
    a = ("În anul %d au avut loc un total de %d tentative de lansări orbitale, din care %d eșecuri." % (year, succ_country(year)[11]+fail_country(year)[11], fail_country(year)[11]))
    return(a)

In [14]:
def launch_stat(year):
# ylaunch
	i = 0
	field_names = ["ID", "Dată (UTC)", "Lansator", "Serie", "Satelit (misiune)", "Or", "Centru", "R"]
	data = []

	while i<ldata_rows:
		if str(year) in str(ldata[i][2][0:7]):
			data.append([ldata[i][0].strip(), ldata[i][2] if ":" not in ldata[i][2] else ldata[i][2][:-3], ldata[i][3] if len(ldata[i][4])<2 else ldata[i][3]+" / "+ldata[i][4], ldata[i][5], ldata[i][6] if ldata[i][6].strip() == ldata[i][7].strip() else ldata[i][6].strip() +" ("+ldata[i][7].strip()+")", ldata[i][26], ldata[i][10] +"+"+ ldata[i][11], ldata[i][27]])
		i+=1
	df_ls = pd.DataFrame(data, columns=field_names)
	return(df_ls.to_markdown(index=False))

In [15]:
def launch_stat_list(year):
# zlaunch

	l_tot_country = [0] * 12
	l_fail_country = fail_country(year)
	l_succ_country = succ_country(year)
	field_names = ["Țara", "Tentative", "Reușite", "Eșecuri"]
	data = []

	i = 0
	while i<10:
		l_tot_country[i] = l_succ_country[i] + l_fail_country[i]
		i+=1
    
	i = 0
	while i<11:
		if l_tot_country[i]>0: data.append([country[i], l_succ_country[i]+l_fail_country[i], l_succ_country[i], l_fail_country[i]])
		i+=1
	df_lsl = pd.DataFrame(data, columns=field_names)

	return(df_lsl.to_markdown(index=False))

In [16]:
def launch_stat_graph(year):

	%matplotlib inline

	l_tot_country = [0] * 12
	l_fail_country = fail_country(year)
	l_succ_country = succ_country(year)
    
	i = 0
	while i<10:
		l_tot_country[i] = l_succ_country[i] + l_fail_country[i]
		i+=1

	list_countries = ["US", "CN", "RU", "EU", "JP", "IN", "IR", "IL", "KR", "KP"]
	list_slaunches = [l_succ_country[0], l_succ_country[2], l_succ_country[1], l_succ_country[3], l_succ_country[4], l_succ_country[5], l_succ_country[6], l_succ_country[7], l_succ_country[8], l_succ_country[9]]
	list_tlaunches = [l_tot_country[0], l_tot_country[2], l_tot_country[1], l_tot_country[3], l_tot_country[4], l_tot_country[5], l_tot_country[6], l_tot_country[7], l_tot_country[8], l_tot_country[9]]

	plt.bar(list_countries, list_tlaunches, color="r", label="Lansări eșuate")
	plt.bar(list_countries, list_slaunches, color="b", label="Lansări reușite")
	plt.title("Lansări orbitale %s" % str(year))
	plt.legend()
	plt.show()
	plt.close()

In [26]:
def all_succ_launches():
    i = curr_year
    data = []
    field_names = ["An", "Total", "SUA", "RU", "CN", "EU", "JP", "IN", "IR", "IL", "KP", "KR", "BR"]
    while i>1956:
        l_succ_country = succ_country(i)
        data.append([i, l_succ_country[11], l_succ_country[0], l_succ_country[1], l_succ_country[2], l_succ_country[3], l_succ_country[4], l_succ_country[5], l_succ_country[6], l_succ_country[7], l_succ_country[8], l_succ_country[9], l_succ_country[10]])
        i-=1
    df_als = pd.DataFrame(data, columns=field_names)
    return(df_als.to_markdown(index=False))

def all_fail_launches():
    i = curr_year
    data = []
    field_names = ["An", "Total", "SUA", "RU", "CN", "EU", "JP", "IN", "IR", "IL", "KP", "KR", "BR"]
    while i>1956:
        l_fail_country = fail_country(i)
        data.append([i, l_fail_country[11], l_fail_country[0], l_fail_country[1], l_fail_country[2], l_fail_country[3], l_fail_country[4], l_fail_country[5], l_fail_country[6], l_fail_country[7], l_fail_country[8], l_fail_country[9], l_fail_country[10]])
        i-=1
    df_alf = pd.DataFrame(data, columns=field_names)
    return(df_alf.to_markdown(index=False))

def all_succ_launches_matrix():
    i = 1957
    all_succ = [0 for i in range(curr_year-1957+1)] 
    while i<=curr_year:
        l_succ_country = succ_country(i)
        all_succ[i-1957] = l_succ_country[11]
        i+=1
    return(all_succ)

def all_fail_launches_matrix():
    i = 1957
    all_fail = [0 for i in range(curr_year-1957+1)] 
    while i<=curr_year:
        l_fail_country = fail_country(i)
        all_fail[i-1957] = l_fail_country[11]
        i+=1
    return(all_fail)

def all_tot_launches_matrix():
    a = all_succ_launches_matrix()
    b = all_fail_launches_matrix()
    all_tot = [0 for i in range(curr_year-1957+1)] 
    i = 0
    while i < curr_year-1957+1:
        all_tot[i] = a[i] + b[i]
        i+=1
    return(all_tot)

table_all_succ = all_succ_launches()
total_launches = all_tot_launches_matrix()
total_failed = all_fail_launches_matrix()
total_succ = all_succ_launches_matrix()
%store table_all_succ
%store total_launches
%store total_failed
%store total_succ
%store table_all_succ

Stored 'table_all_succ' (str)
Stored 'total_launches' (list)
Stored 'total_failed' (list)
Stored 'total_succ' (list)
Stored 'table_all_succ' (str)


In [18]:
def racheta(nume):
    i = 0
    j = 0
    field_names = ["ID", "Dată (UTC)", "Lansator", "Serie", "Satelit (misiune)", "Or", "Centru", "R"]
    data = []
    r_launches = 0
    r_fail = 0
    while i<ldata_rows:
        if nume in ldata[i][3]: 
            data.append([ldata[i][0].strip(), ldata[i][2] if ":" not in ldata[i][2] else ldata[i][2][:-3], ldata[i][3], ldata[i][5], ldata[i][6] if (((ldata[i][6].strip() == ldata[i][7].strip()) or (ldata[i][7].strip == "-"))) else ldata[i][6].strip() +" ("+ldata[i][7].strip()+")", ldata[i][26], ldata[i][10] +"+"+ ldata[i][11], ldata[i][27]])
            r_launches+=1
            if "F" in ldata[i][27]: 
                r_fail+=1
                j = 0      
            j+=1
        i+=1
    line6 = ""
    launches_since_fail = j-1
    
    df_racheta = pd.DataFrame(data, columns=field_names)
    
    if r_launches == 0: 
        rate_succ = 100 
    else: 
        rate_succ = 100 - (r_fail / r_launches *100)
    
    line1 = ("Lista lansărilor orbitale pentru racheta %s\n\n" % nume)
    line3 = ("Număr total de lansări %s: %d\n" % (nume, r_launches))
    line4 = ("Număr de eșecuri %s: %d\n" % (nume, r_fail))
    line5 = ("Rată de succes %s: %.2f%%\n" % (nume, rate_succ))
    if r_fail > 0: line6 = ("Număr de lansări reușite de la ultimul eșec: %d\n" % (launches_since_fail))
    
    r_content = line3 + line4 + line5 + line6 +"\n"+ line1
    return (r_content, df_racheta.to_markdown(index=False))

In [19]:
def active_rockets():
#    list_active_rockets = []
    i = 1
    field_names = ["Rachetă", "O.", "Cea mai recentă lansare"]
    data = []
    while i<ldata_rows:
        if ldata[-i][3] not in list_active_rockets: 
#            list_active_rockets.append(ldata[-i][3])
            data.append([ldata[-i][3], ldata[-i][26], ldata[-i][2] if ":" not in ldata[-i][2] else ldata[-i][2][:-3]])
        i+=1
    df_active_rockets = pd.DataFrame(data, columns=field_names)
    return(df_active_rockets.to_markdown(index=False))

In [20]:
def recent_launches(n):
    field_names = ["ID", "Dată (UTC)", "Lansator", "Serie", "Satelit (misiune)", "Or", "Centru"]
    data = []
    i = 1
    while i<n:
        data.append([ldata[-i][0].strip(), ldata[-i][2] if ":" not in ldata[-i][2] else ldata[-i][2][:-3], ldata[-i][3] if len(ldata[-i][4])<2 else ldata[-i][3]+" / "+ldata[-i][4], ldata[-i][5], ldata[-i][6] if ldata[-i][6].strip() == ldata[-i][7].strip() else ldata[-i][6].strip() +" ("+ldata[-i][7].strip()+")", ldata[-i][26], ldata[-i][10] +"+"+ ldata[-i][11]])
        i+=1
    df_recent_launches = pd.DataFrame(data, columns=field_names)
    return(df_recent_launches.to_markdown(index=False))

In [21]:
def show_starlink():
    field_names = ["ID", "Dată (UTC)", "Lansator", "Serie", "Satelit (misiune)", "Centru"]
    data = []
    i = 0
    j = 0
    while i < ldata_rows:
        if ("Starlink" in ldata[i][6]) or ("Starlink" in ldata[i][7]):
            j+=1
            data.append([ldata[i][0].strip(), ldata[i][2] if ":" not in ldata[i][2] else ldata[i][2][:-3], ldata[-i][3] if len(ldata[i][4])<2 else ldata[i][3]+" / "+ldata[i][4], ldata[i][5], ldata[i][6] if ldata[i][6].strip() == ldata[i][7].strip() else ldata[i][6].strip() +" ("+ldata[i][7].strip()+")", ldata[i][10] +"+"+ ldata[i][11]])
        i+=1
    today = date.today()
    a =  ("Până la data %s, au ajut loc %d lansări Starlink\n" % (str(today), j))
    df_show_starlink = pd.DataFrame(data, columns=field_names)
    print(a)
    return(a, df_show_starlink.to_markdown(index=False))

list_starlink = show_starlink()
%store list_starlink

Până la data 2022-11-08, au ajut loc 65 lansări Starlink

Stored 'list_starlink' (tuple)


In [22]:
zlaunch = []
i = 1957
while i<=curr_year:
    zlaunch.append(launch_stat_list(i))
    i+=1
%store zlaunch

Stored 'zlaunch' (list)


In [23]:
ylaunch = []
i = 1957
while i<=curr_year:
    ylaunch.append(launch_stat(i))
    i+=1
%store ylaunch

Stored 'ylaunch' (list)


In [24]:
mlaunch = []
i = 1957
while i<=curr_year:
    mlaunch.append(preambul(i))
    i+=1
#print(mlaunch)
%store mlaunch

Stored 'mlaunch' (list)


In [25]:
most_recent_launches = recent_launches(12)
%store most_recent_launches

falcon9 = racheta("Falcon 9")
%store falcon9

falconh = racheta("Falcon Heavy")
%store falconh

ariane5 = racheta("Ariane 5")
%store ariane5

vegac = racheta("Vega")
%store vegac

atlasv = racheta("Atlas V")
%store atlasv

delta4 = racheta("Delta 4M")
%store delta4

delta4h = racheta("Delta 4 Heavy")
%store delta4h

electron = racheta("Electron")
%store electron

fireflya = racheta("Firefly")
%store fireflya

launcherone = racheta("LauncherOne")
%store launcherone

astra = racheta("Astra")
%store astra

antares = racheta("Antares")
%store antares

minotaur = racheta("Minotaur")
%store minotaur

pegasus = racheta("Pegasus")
%store pegasus

hii = racheta("H-II")
%store hii

epsilon = racheta("Epsilon")
%store epsilon

cz2c = racheta("Chang Zheng 2C")
%store cz2c

cz2d = racheta("Chang Zheng 2D")
%store cz2d

cz2f = racheta("Chang Zheng 2F")
%store cz2f

cz3b = racheta("Chang Zheng 3B")
%store cz3b

cz3c = racheta("Chang Zheng 3C")
%store cz3c

cz4b = racheta("Chang Zheng 4B")
%store cz4b

cz4c = racheta("Chang Zheng 4C")
%store cz4c

cz5 = racheta("Chang Zheng 5")
%store cz5

cz6 = racheta("Chang Zheng 6")
%store cz6

cz7 = racheta("Chang Zheng 7")
%store cz7

cz8 = racheta("Chang Zheng 8")
%store cz8

cz11 = racheta("Chang Zheng 11")
%store cz11

kz1a = racheta("Kuaizhou-1A")
%store kz1a

kz11 = racheta("Kuaizhou-11")
%store kz11

gushenxing = racheta("Gushenxing")
%store gushenxing

lijian = racheta("Lijian-1")
%store lijian

sq1 = racheta("Shuang Quxian")
%store sq1

jielong = racheta("Jielong")
%store jielong

osm1 = racheta ("OS-M1")
%store osm1

zhuque1 = racheta("Zhuque-1")
%store zhuque1

kt2 = racheta("KT-2")
%store kt2

soyuz21 = racheta("Soyuz-2.1")
%store soyuz21

soyuz21v = racheta("Soyuz-2.1v")
%store soyuz21v

angara12 = racheta("Angara-1.2")
%store angara12

angaraa5 = racheta("Angara A5")
%store angaraa5

protonm = racheta("Proton-M")
%store protonm

kwangmyongsong = racheta("Kwangmyongsong")
%store kwangmyongsong

safir = racheta("Safir")
%store safir

simorgh = racheta("Simorgh")
%store simorgh

qased = racheta("Qased")
%store qased

shavit = racheta("Shavit")
%store shavit

nuri = racheta("Nuri")
%store nuri

gslvmk3 = racheta("GSLV Mk. 3")
%store gslvmk3

gslvmk2 = racheta("GSLV Mk. 2")
%store gslvmk2

pslv = racheta("PSLV")
%store pslv

sslv = racheta("SSLV")
%store sslv

Stored 'most_recent_launches' (str)
Stored 'falcon9' (tuple)
Stored 'falconh' (tuple)
Stored 'ariane5' (tuple)
Stored 'vegac' (tuple)
Stored 'atlasv' (tuple)
Stored 'delta4' (tuple)
Stored 'delta4h' (tuple)
Stored 'electron' (tuple)
Stored 'fireflya' (tuple)
Stored 'launcherone' (tuple)
Stored 'astra' (tuple)
Stored 'antares' (tuple)
Stored 'minotaur' (tuple)
Stored 'pegasus' (tuple)
Stored 'hii' (tuple)
Stored 'epsilon' (tuple)
Stored 'cz2c' (tuple)
Stored 'cz2d' (tuple)
Stored 'cz2f' (tuple)
Stored 'cz3b' (tuple)
Stored 'cz3c' (tuple)
Stored 'cz4b' (tuple)
Stored 'cz4c' (tuple)
Stored 'cz5' (tuple)
Stored 'cz6' (tuple)
Stored 'cz7' (tuple)
Stored 'cz8' (tuple)
Stored 'cz11' (tuple)
Stored 'kz1a' (tuple)
Stored 'kz11' (tuple)
Stored 'gushenxing' (tuple)
Stored 'lijian' (tuple)
Stored 'sq1' (tuple)
Stored 'jielong' (tuple)
Stored 'osm1' (tuple)
Stored 'zhuque1' (tuple)
Stored 'kt2' (tuple)
Stored 'soyuz21' (tuple)
Stored 'soyuz21v' (tuple)
Stored 'angara12' (tuple)
Stored 'angaraa5' (t